In [3]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sn
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np

In [5]:
data=pd.read_excel('Balance - Tablas (Parte 1 ).xlsx','pacientes',engine='openpyxl')
data.head()

,IdPaciente,AnioNacimiento,IdConsultorio,Ciudad,Estado,Sexo,EstadoCivil,Religion,Ocupacion,Escolaridad,Fecha,Hora,IdInteresado,ProgramaRecomendados,SeguridadSocial,RecordarCita,EstadioValor,Estadio
0,1,1928.0,1,GUADALUPE,NUEVO LEÓN,H,SOLTERÍA,CATOLICA,ALBAÑIL,NINGUNA,2013-02-19,13:55:23,16432,0,imss,1,NaN,NaN
1,2,1971.0,1,GUADALUPE,NUEVO LEÓN,H,MATRIMONIO,NaN,INSTALACION MECANICA,LEER Y ESCRIBIR,2013-02-19,13:55:23,16433,0,IMSS,1,NaN,NaN
2,3,1943.0,1,GUADALUPE,NUEVO LEÓN,M,MATRIMONIO,CATOLICA,HOGAR,PREPARATORIA,2013-02-19,13:55:23,16434,0,IMSS,1,8.8,5.0
3,4,1976.0,1,SANTIAGO,NUEVO LEÓN,H,SOLTERÍA,CATOLICA,EMPLEADO,LICENCIATURA,2013-02-19,13:55:23,16435,0,PRIVADO,1,NaN,NaN
4,5,1958.0,1,GUADALUPE,NUEVO LEÓN,H,MATRIMONIO,CATÓLICA,CASADO,PREPARATORIA,2013-02-19,13:55:23,16436,1,NO SEGURIDAD SOCIAL,1,NaN,NaN


In [11]:
#extracting lst of all patients from balnce table part1 and storing it in patients dataframe
patients=data['IdPaciente']
print(patients.nunique())
print(data.shape[0])

14855
14855


In [21]:

data2 = pd.read_excel('IV_Datos_CMU.xlsx','cm_tickets',parse_dates=['Fecha'],engine='openpyxl')
data2.head()

,IdPaciente,Fecha,AbonoMembresiaInscripcion,CompraFarmacia,CompraTienda
0,3299,2014-07-17,1,0,0
1,3301,2014-07-17,1,0,0
2,3311,2014-07-18,1,0,0
3,3311,2014-07-19,1,0,0
4,3321,2014-07-19,1,0,0


In [22]:
#extracting patients who have buyed or renewed membership 
renewed_patients=data2[data2['AbonoMembresiaInscripcion']==1]
renewed_patients=renewed_patients.iloc[:,:3]
renewed_patients

,IdPaciente,Fecha,AbonoMembresiaInscripcion
0,3299,2014-07-17,1
1,3301,2014-07-17,1
2,3311,2014-07-18,1
3,3311,2014-07-19,1
4,3321,2014-07-19,1
...,...,...,...
77150,12036,2016-12-31,1
77151,12955,2016-12-31,1
77152,12135,2016-12-31,1
77153,5261,2016-12-31,1


In [33]:
# joining these data 
renewed_joined_data=pd.merge(patients,renewed_patients,on='IdPaciente',how='left')
renewed_joined_data.head()

,IdPaciente,Fecha,AbonoMembresiaInscripcion
0,1,2014-09-30,1.0
1,2,NaT,NaN
2,3,NaT,NaN
3,4,NaT,NaN
4,5,2015-06-03,1.0


In [34]:
#filling nan values in AbonoMembresiaInscripcion with 0 and renaming the column to value
renewed_joined_data['AbonoMembresiaInscripcion']=renewed_joined_data['AbonoMembresiaInscripcion'].fillna(0)
renewed_joined_data = renewed_joined_data.rename(columns={'AbonoMembresiaInscripcion': 'value'})
renewed_joined_data.head()

,IdPaciente,Fecha,value
0,1,2014-09-30,1.0
1,2,NaT,0.0
2,3,NaT,0.0
3,4,NaT,0.0
4,5,2015-06-03,1.0


In [35]:
renewed_joined_data.to_csv('renews_membership.csv',header=True)

In [47]:
renewed_joined_data['value'].describe()

count    43711.000000
mean         0.914896
std          0.279040
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: value, dtype: float64

In [40]:
#extracting patients who have canceled membership
cancelled_patients=data2[data2['AbonoMembresiaInscripcion']==0]
cancelled_patients=cancelled_patients.iloc[:,:3]
cancelled_patients

,IdPaciente,Fecha,AbonoMembresiaInscripcion
12,3376,2014-07-25,0
32,3522,2014-08-11,0
33,3487,2014-08-12,0
34,3491,2014-08-12,0
35,3135,2014-08-12,0
...,...,...,...
77132,13817,2016-12-30,0
77133,8874,2016-12-30,0
77135,14931,2016-12-30,0
77136,15068,2016-12-30,0


In [41]:
# joining this data with all patients
cancelled_joined_data=pd.merge(patients,cancelled_patients,on='IdPaciente',how='left')
cancelled_joined_data.head()

,IdPaciente,Fecha,AbonoMembresiaInscripcion
0,1,2014-12-01,0.0
1,2,NaT,NaN
2,3,NaT,NaN
3,4,NaT,NaN
4,5,NaT,NaN


In [42]:
#0 in AbonoMembresiaInscripcion is 1 in cancelled membership value
cancelled_joined_data['AbonoMembresiaInscripcion']=cancelled_joined_data['AbonoMembresiaInscripcion'].replace(0,1)
cancelled_joined_data

,IdPaciente,Fecha,AbonoMembresiaInscripcion
0,1,2014-12-01,1.0
1,2,NaT,NaN
2,3,NaT,NaN
3,4,NaT,NaN
4,5,NaT,NaN
...,...,...,...
43546,15064,NaT,NaN
43547,15065,NaT,NaN
43548,15066,NaT,NaN
43549,15067,NaT,NaN


In [43]:
#filling nan values in AbonoMembresiaInscripcion with 0 and renaming the column to value
cancelled_joined_data['AbonoMembresiaInscripcion']=cancelled_joined_data['AbonoMembresiaInscripcion'].fillna(0)
cancelled_joined_data = cancelled_joined_data.rename(columns={'AbonoMembresiaInscripcion': 'value'})
cancelled_joined_data.head()

,IdPaciente,Fecha,value
0,1,2014-12-01,1.0
1,2,NaT,0.0
2,3,NaT,0.0
3,4,NaT,0.0
4,5,NaT,0.0


In [44]:
cancelled_joined_data.to_csv('cancels_membership.csv',header=True)

In [46]:
cancelled_joined_data['value'].describe()

count    43551.000000
mean         0.852931
std          0.354179
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: value, dtype: float64